In [43]:
import os
import sys
sys.path.append(".")
sys.path.append("..")
import imodels
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from imodels.tree.rf_plus.rf_plus.rf_plus_models import RandomForestPlusRegressor, RandomForestPlusClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, mean_squared_error, r2_score, average_precision_score
from imodels.tree.rf_plus.feature_importance.rfplus_explainer import *
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from feature_importance.scripts.simulations_util import *
from scripts.competing_methods_local import *
from rbo_implementation import rbo_dict
import sys
sys.path.append('..')
sys.path.append('../..')
sys.path.append('.')
sys.path.append('./scripts')
from competing_methods_local import *
from simulations_util import *
from sklearn.linear_model import ElasticNetCV  
from sklearn import model_selection
from sklearn.svm import SVC
from imodels.importance import RandomForestPlusRegressor as RandomForestPlusRegressorOld
import dice_ml
from sklearn.neighbors import NearestNeighbors

In [44]:
def apply_splitting_strategy(X: np.ndarray,
                             y: np.ndarray,
                             splitting_strategy: str,
                             split_seed: str):
    if splitting_strategy in {'train-test-lowdata', 'train-tune-test-lowdata'}:
        test_size = 0.90
    elif splitting_strategy == "train-test":
        test_size = 0.33
    else:
        test_size = 0.2

    X_train, X_test, y_train, y_test = model_selection.train_test_split(
        X, y, test_size=test_size, random_state=split_seed)
    X_tune = None
    y_tune = None

    if splitting_strategy in {'train-tune-test', 'train-tune-test-lowdata'}:
        X_train, X_tune, y_train, y_tune = model_selection.train_test_split(
            X_train, y_train, test_size=0.2, random_state=split_seed)

    return X_train, X_tune, X_test, y_train, y_tune, y_test

In [ ]:
def num_features_distance(vec1, vec2):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    return np.sum(vec1 != vec2)

def proximity_score_distance(vec1, vec2, continous_idx, data):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    continous_idx = np.array(continous_idx)
    distance = 0
    data = np.array(data)
    mad_values = np.median(np.abs(data[:, continous_idx] - np.median(data[:, continous_idx], axis=0)), axis=0)
    # mad_values[mad_values == 0] = 1e-6  # Avoid division by zero

    for i in range(len(vec1)):
        if i in continous_idx:
            distance += np.abs(vec1[i] - vec2[i]) / mad_values[np.where(continous_idx == i)[0][0]]
        else:
            distance += vec1[i] != vec2[i]

    return distance

In [ ]:
def counterfactual_feature_importance(train_feature_importance, train_pred, test_feature_importance, test_pred):
    scaler = MinMaxScaler()
    filtered_indices = np.where(train_pred != test_pred)[0]
    train_feature_importance_filtered = train_feature_importance[filtered_indices]
    train_feature_importance_filtered = scaler.fit_transform(train_feature_importance_filtered)
    test_feature_importance = scaler.transform(test_feature_importance.reshape(1, -1)).flatten()
    
    k = 5
    distances = []
    for i, train_instance in enumerate(train_feature_importance_filtered):
        dist = proximity_score_distance(train_instance, test_feature_importance)
        distances.append((i, dist))
    distances.sort(key=lambda x: x[1])
    knn_indices = [idx for idx, _ in distances[:k]]
    
    return filtered_indices[knn_indices]

def evaluate_counterfactuals(X_train, y_train, indices, explained_instance, model):
    predictions = []
    numm_features = []
    proximity_score = []
    for idx in indices:
        counterfactual = X_train[idx]
        prediction = model.predict(counterfactual.reshape(1, -1))
        predictions.append(prediction)
        numm_features.append(num_features_distance(explained_instance, counterfactual))
        proximity_score.append(proximity_score_distance(explained_instance, counterfactual, model.continuous_idx, X_train))
    return predictions, numm_features, proximity_score

SyntaxError: incomplete input (3770735116.py, line 18)

In [81]:
proximity_score_distance(X_train.values[0], X_train.values[3], [0,3,4], X_train)

45.30769231915474

In [71]:
data = X_train.values

In [73]:
X_train.columns

Index(['age', 'priors_count', 'days_b_screening_arrest', 'c_jail_time',
       'juv_fel_count', 'juv_other_count', 'juv_misd_count',
       'c_charge_degree:F', 'c_charge_degree:M', 'race:African-American',
       'race:Asian', 'race:Caucasian', 'race:Hispanic', 'race:Native_American',
       'race:Other', 'age_cat:25_-_45', 'age_cat:Greater_than_45',
       'age_cat:Less_than_25', 'sex:Female', 'sex:Male'],
      dtype='object')

In [76]:
X_train['days_b_screening_arrest'].value_counts()

days_b_screening_arrest
-1.0     448
 0.0     149
-2.0      14
-3.0       7
-7.0       4
-22.0      3
-18.0      3
-28.0      3
-12.0      3
-10.0      3
-23.0      3
-6.0       3
-26.0      2
-8.0       2
-5.0       2
-4.0       2
-11.0      2
-25.0      2
-24.0      2
-13.0      2
-29.0      2
-20.0      1
 26.0      1
-16.0      1
 22.0      1
 2.0       1
 16.0      1
 18.0      1
-17.0      1
-15.0      1
Name: count, dtype: int64

In [74]:
continous_idx = [0, 2,3]

In [75]:
np.median(np.abs(data[:, continous_idx] - np.median(data[:, continous_idx], axis=0)), axis=0)

array([6.5, 0. , 1. ], dtype=float32)

In [62]:
X_train.values[0]

array([27.,  2., -1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,
        0.,  0.,  1.,  0.,  0.,  0.,  1.], dtype=float32)

In [67]:
X_train.values[10] == X_train.values[0]

array([False, False,  True,  True,  True,  True,  True, False, False,
       False,  True,  True, False,  True,  True, False,  True, False,
        True,  True])

In [68]:
num_features_distance(X_train.values[0], X_train.values[10])

8

In [ ]:
def check_prediction_counterfactuals(model, explained_instance, counterfactuals):
    explained_instance = explained_instance.to_numpy()
    counterfactuals = counterfactuals.to_numpy()
    explained_instance_pred = model.predict(explained_instance)
    counterfactuals_preds = model.predict(counterfactuals)
    return np.mean(explained_instance_pred != counterfactuals_preds)

def check_num_features_counterfactuals(explained_instance, counterfactuals):
    explained_instance = explained_instance.to_numpy()
    counterfactuals = counterfactuals.to_numpy()
    return np.mean((explained_instance != counterfactuals).sum(axis=1))

def proximity_score(counterfactuals, x, continous_features_index):
    x = x.to_numpy().flatten()
    counterfactuals = counterfactuals.to_numpy()
    cont_proximity = 0
    cat_proximity = 0
    for i in range(counterfactuals.shape[0]):
        for j in range(counterfactuals.shape[1]):
            if j in continous_features_index:
                cont_proximity += np.abs(counterfactuals[i][j] - x[j])
            else:
                cat_proximity += counterfactuals[i][j] != x[j]
    return cont_proximity / counterfactuals.shape[0], cat_proximity / counterfactuals.shape[0]

# def median_absolute_deviation(arr):
#     median = np.median(arr)
#     return np.median(np.abs(arr - median))

# def proximity_score(counterfactuals, x, continous_features_index, mad_values):
#     total_proximity = 0
#     for i in range(counterfactuals.shape[0]):
#         proximity = 0
#         for j in range(counterfactuals.shape[1]):
#             if j in continous_features_index:
#                 proximity += np.abs(counterfactuals[i][j] - x[j]) / mad_values[j]
#             else:
#                 proximity += counterfactuals[i][j] != x[j]
#         total_proximity += proximity
#     return total_proximity / counterfactuals.shape[0]

In [3]:
X, y, feature_names = imodels.get_clean_dataset('compas_two_year_clean', data_source='imodels')
X = pd.DataFrame(X, columns=feature_names)
# set random seed
np.random.seed(0)
keep_idx = np.random.choice(X.shape[0], 1000, replace=False)
X = X.iloc[keep_idx, :]
y = y[keep_idx]
continuous_columns = ['age', 'days_b_screening_arrest', 'c_jail_time']#[col for col in X.select_dtypes(include=['number']).columns if X[col].nunique() > 5]
continuous_columns_idx = [X.columns.get_loc(col) for col in continuous_columns]
X_train, X_tune, X_test, y_train, y_tune, y_test = apply_splitting_strategy(X, y, "train-test", 0)

In [5]:
est = RandomForestClassifier(n_estimators=100, min_samples_leaf=1, max_features="sqrt", random_state=1)
est.fit(pd.DataFrame(X_train, columns=feature_names), y_train)
# rf_plus_elastic = RandomForestPlusClassifier(rf_model=est, prediction_model=LogisticRegressionCV(penalty='elasticnet', l1_ratios=[0.1,0.5,0.99], solver = 'saga', cv=3, n_jobs=-1, tol=5e-4, max_iter=2000, random_state=0))
# rf_plus_elastic.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [29]:
train_pred = est.predict(X_train)
test_pred = est.predict(X_test)

In [11]:
treeshap_train, treeshap_test = tree_shap_evaluation_RF_retrain(X_train, y_train, X_test, est, mode=None)

In [59]:
pd.DataFrame(treeshap_train)[7].value_counts()

7
-0.016215    4
-0.011032    3
-0.005999    3
-0.013754    2
 0.026765    2
            ..
-0.037718    1
 0.058851    1
-0.007925    1
-0.016478    1
 0.003544    1
Name: count, Length: 646, dtype: int64

In [47]:
from sklearn.preprocessing import MinMaxScaler

def counterfactual_feature_importance(train_feature_importance, train_pred, test_feature_importance, test_pred):
    scaler = MinMaxScaler()
    filtered_indices = np.where(train_pred != test_pred)[0]
    train_feature_importance_filtered = train_feature_importance[filtered_indices]
    train_feature_importance_filtered = scaler.fit_transform(train_feature_importance_filtered)
    test_feature_importance = scaler.transform(test_feature_importance.reshape(1, -1))
    k = 5
    knn = NearestNeighbors(n_neighbors=k, metric='euclidean')
    knn.fit(train_feature_importance_filtered)
    _, indices = knn.kneighbors(test_feature_importance)
    return filtered_indices[indices[0]]

In [49]:
counterfactual_feature_importance(treeshap_train, train_pred, treeshap_test[0], test_pred[0])

array([425, 172, 321, 296, 582])

In [48]:
treeshap_test

array([[-8.39171740e-02, -3.45642654e-02, -3.57459360e-02, ...,
        -2.92675001e-02,  3.89679694e-03,  7.19917008e-03],
       [-1.16146027e-01,  2.39265564e-01,  1.76552753e-04, ...,
        -1.09831213e-02,  7.11622044e-03,  5.06678400e-04],
       [ 1.28806052e-01, -4.72735448e-02, -1.78254913e-03, ...,
         6.77213773e-02,  4.95080455e-04, -7.61628400e-04],
       ...,
       [-7.24062439e-02,  1.96828879e-01,  2.08876223e-02, ...,
        -9.60288413e-03,  1.18707784e-02,  8.86892096e-03],
       [ 1.69673992e-03, -6.15842940e-02, -5.69434952e-02, ...,
        -1.65560242e-02,  6.66870129e-03,  1.17295004e-02],
       [-2.33290077e-01,  4.72234720e-02,  2.60568002e-03, ...,
        -2.82562380e-02, -1.53995236e-02, -1.20906533e-02]])

In [31]:
from sklearn.neighbors import NearestNeighbors
i = 0
test_instance_fi = treeshap_test[i]
test_instance_pred = test_pred[i]
filtered_indices = np.where(train_pred != test_instance_pred)[0]
treeshap_train_filtered = treeshap_train[filtered_indices]
test_instance_fi = test_instance_fi.reshape(1, -1)
k = 5
knn = NearestNeighbors(n_neighbors=k, metric='euclidean')
knn.fit(treeshap_train_filtered)

NearestNeighbors(metric='euclidean')

In [37]:
_, indices = knn.kneighbors(test_instance_fi)

In [38]:
indices

array([[285, 318, 276, 313, 307]])

In [39]:
filtered_indices[indices[0]]

array([587, 653, 572, 646, 635])

In [41]:
X_train.values[filtered_indices[indices[0]]]

array([[44.,  0., -1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  1.,  0.,  0.,  0.,  1.],
       [44.,  0., -1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  1.,  0.,  0.,  0.,  1.],
       [37.,  0., -1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.,  1.,  0.,  0.,  0.,  1.],
       [45.,  0., -1.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  1.,  0.,  0.,  1.],
       [51.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  1.,  0.,  0.,  1.]], dtype=float32)

In [42]:
X_test.values[i]

array([36.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,
        0.,  0.,  1.,  0.,  0.,  0.,  1.], dtype=float32)

In [ ]:
_, indices = knn.kneighbors(test_instance_fi)

# Get the indices of the k-nearest neighbors (map back to original indices)
knn_indices = filtered_indices[indices[0]]
print("Indices of k-nearest neighbors:", knn_indices)

# Get the actual nearest neighbor instances
knn_instances = treeshap_train[knn_indices]
print("Nearest neighbor instances:\n", knn_instances)

In [33]:
treeshap_train_filtered.shape

(325, 20)

In [25]:
treeshap_train.shape

(670, 20)

In [27]:
test_instance_fi.shape

(20,)

In [23]:
np.linalg.norm(treeshap_train - test_instance_fi, axis=1).shape

(670,)

In [21]:
test_instance_pred

array([0])

In [20]:
test_instance_fi

array([-0.08391717, -0.03456427, -0.03574594, -0.03602561, -0.00162602,
       -0.00337615,  0.00112197,  0.00152203, -0.00240882,  0.00688912,
       -0.0002767 , -0.00316656,  0.0032288 , -0.00012124,  0.00212467,
       -0.00943394,  0.01145482, -0.0292675 ,  0.0038968 ,  0.00719917])

In [13]:
treeshap_test[1]

array([-1.16146027e-01,  2.39265564e-01,  1.76552753e-04,  9.32144316e-02,
       -1.08171996e-04, -3.64287987e-03,  2.02924587e-03, -2.45137744e-03,
        8.30047073e-03, -2.56660885e-02, -1.66203720e-04, -1.34213143e-02,
        4.23440293e-03, -1.02758225e-04, -1.89012548e-04,  8.42603842e-03,
       -4.28851868e-02, -1.09831213e-02,  7.11622044e-03,  5.06678400e-04])

### DICE

In [35]:
# Get DICE explanations
pd.set_option('display.max_columns', None)
train_df = pd.DataFrame(X_train, columns=feature_names)
train_df['target'] = y_train
test_df = pd.DataFrame(X_test, columns=feature_names)

d = dice_ml.Data(dataframe=train_df, continuous_features=continuous_columns, outcome_name='target')
m = dice_ml.Model(model=est, backend="sklearn")
exp = dice_ml.Dice(d, m)
explained_index = 0
query_instance = test_df[explained_index:explained_index+1]
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=5, desired_class="opposite")
dice_exp.cf_examples_list[0].final_cfs_df
#dice_exp.cf_examples_list[0].final_cfs_df.apply(lambda row: (train_df == row).all(axis=1).any(), axis=1)

  0%|          | 0/1 [00:00<?, ?it/s]/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/dice_ml/explainer_interfaces/dice_random.py:116: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5.0' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  candidate_cfs.at[k, selected_features[k][0]] = random_instances.at[k, selected_features[k][0]]
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/dice_ml/explainer_interfaces/dice_random.py:116: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  candidate_cfs.at[k, selected_features[k][0]] = random_instances.at[k, selected_features[k][0]]
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/

,age,priors_count,days_b_screening_arrest,c_jail_time,juv_fel_count,juv_other_count,juv_misd_count,c_charge_degree:F,c_charge_degree:M,race:African-American,race:Asian,race:Caucasian,race:Hispanic,race:Native_American,race:Other,age_cat:25_-_45,age_cat:Greater_than_45,age_cat:Less_than_25,sex:Female,sex:Male,target
0,36.0,1.0,0.0,716.299988,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0
1,36.0,1.0,0.0,732.299988,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0
2,21.0,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1
3,31.5,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1
4,36.0,9.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1


In [36]:
test_df.iloc[explained_index:explained_index+1]

,age,priors_count,days_b_screening_arrest,c_jail_time,juv_fel_count,juv_other_count,juv_misd_count,c_charge_degree:F,c_charge_degree:M,race:African-American,race:Asian,race:Caucasian,race:Hispanic,race:Native_American,race:Other,age_cat:25_-_45,age_cat:Greater_than_45,age_cat:Less_than_25,sex:Female,sex:Male
4529,36.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [37]:
dice_counterfactuals = dice_exp.cf_examples_list[0].final_cfs_df.drop(columns=['target'])
print(check_prediction_counterfactuals(est, query_instance, dice_counterfactuals))
print(check_num_features_counterfactuals(query_instance, dice_counterfactuals))
print(proximity_score(dice_counterfactuals, query_instance, continuous_columns_idx))

0.6
1.2
(293.6199951171875, 0.4)


/accounts/projects/binyu/zhongyuan_liang/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/accounts/projects/binyu/zhongyuan_liang/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


### Psi X - support vector

In [38]:
psi_x_train = []
for i in range(len(rf_plus_elastic.transformers_)):
    psi_x_train.append(rf_plus_elastic.transformers_[i].transform(X_train.values).get_all_data())

psi_x_test = []
for i in range(len(rf_plus_elastic.transformers_)):
    psi_x_test.append(rf_plus_elastic.transformers_[i].transform(X_test.values).get_all_data())

psi_svc = []
for i in range(len(psi_x_train)):
    svc = SVC(kernel='linear', C=10)
    svc.fit(psi_x_train[i], y_train)
    psi_svc.append(svc)

In [39]:
closest_sv_index = []
for i in range(len(psi_svc)):
    svm = psi_svc[i]
    
    support_indices = svm.support_
    support_vectors = psi_x_train[i][support_indices]
    
    support_labels = np.sign(svm.decision_function(support_vectors))
    test_label = np.sign(svm.decision_function(psi_x_test[i][0].reshape(1, -1)))
    
    opposite_indices = support_indices[support_labels != test_label]
    
    opposite_support_vectors = psi_x_train[i][opposite_indices]
    distances = np.linalg.norm(opposite_support_vectors - psi_x_test[i][0], axis=1)
    min_index = opposite_indices[np.argmin(distances)]
    closest_sv_index.append(min_index)

In [40]:
unique_elements, counts = np.unique(closest_sv_index, return_counts=True)
element_count_pairs = list(zip(unique_elements, counts))
sorted_element_count_pairs = sorted(element_count_pairs, key=lambda x: x[1], reverse=True)
sorted_counts_dict = dict(sorted_element_count_pairs)

print(sorted_counts_dict)

{280: 9, 501: 6, 521: 5, 129: 4, 321: 4, 60: 3, 172: 3, 312: 3, 492: 3, 525: 3, 600: 3, 123: 2, 140: 2, 194: 2, 245: 2, 251: 2, 276: 2, 290: 2, 334: 2, 370: 2, 425: 2, 451: 2, 13: 1, 15: 1, 51: 1, 73: 1, 86: 1, 89: 1, 113: 1, 114: 1, 135: 1, 191: 1, 200: 1, 260: 1, 267: 1, 277: 1, 281: 1, 299: 1, 344: 1, 390: 1, 401: 1, 415: 1, 424: 1, 487: 1, 489: 1, 539: 1, 559: 1, 570: 1, 572: 1, 577: 1, 604: 1, 616: 1, 620: 1, 658: 1}


In [41]:
unique_elements, counts = np.unique(closest_sv_index, return_counts=True)
df_counts = pd.DataFrame({'Index': unique_elements, 'Count': counts})
df_top5 = df_counts.sort_values(by='Count', ascending=False).head(5)
X_top5 = X_train.iloc[df_top5['Index'].values]

In [42]:
X_top5

,age,priors_count,days_b_screening_arrest,c_jail_time,juv_fel_count,juv_other_count,juv_misd_count,c_charge_degree:F,c_charge_degree:M,race:African-American,race:Asian,race:Caucasian,race:Hispanic,race:Native_American,race:Other,age_cat:25_-_45,age_cat:Greater_than_45,age_cat:Less_than_25,sex:Female,sex:Male
4257,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4416,41.0,3.0,-1.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4106,39.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4254,31.0,1.0,0.0,-1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
5283,32.0,1.0,-1.0,21.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [43]:
test_df.iloc[explained_index:explained_index+1]

,age,priors_count,days_b_screening_arrest,c_jail_time,juv_fel_count,juv_other_count,juv_misd_count,c_charge_degree:F,c_charge_degree:M,race:African-American,race:Asian,race:Caucasian,race:Hispanic,race:Native_American,race:Other,age_cat:25_-_45,age_cat:Greater_than_45,age_cat:Less_than_25,sex:Female,sex:Male
4529,36.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [44]:
print(check_prediction_counterfactuals(est, query_instance, X_top5))
print(check_num_features_counterfactuals(query_instance, X_top5))
print(proximity_score(X_top5, query_instance, continuous_columns_idx))

1.0
3.4
(10.8, 1.0)


/accounts/projects/binyu/zhongyuan_liang/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/accounts/projects/binyu/zhongyuan_liang/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


### Psi X - support vector & Groundtruth

In [45]:
closest_sv_index = []
for i in range(len(psi_svc)):
    svm = psi_svc[i]
    
    support_indices = svm.support_
    support_vectors = psi_x_train[i][support_indices]
    
    support_labels = y_train[support_indices]
    test_label = y_test[explained_index]
    
    opposite_indices = support_indices[support_labels != test_label]
    
    opposite_support_vectors = psi_x_train[i][opposite_indices]
    distances = np.linalg.norm(opposite_support_vectors - psi_x_test[i][0], axis=1)
    min_index = opposite_indices[np.argmin(distances)]
    closest_sv_index.append(min_index)

In [46]:
unique_elements, counts = np.unique(closest_sv_index, return_counts=True)
element_count_pairs = list(zip(unique_elements, counts))
sorted_element_count_pairs = sorted(element_count_pairs, key=lambda x: x[1], reverse=True)
sorted_counts_dict = dict(sorted_element_count_pairs)

print(sorted_counts_dict)

{600: 22, 280: 11, 521: 11, 370: 8, 172: 6, 129: 5, 86: 4, 321: 4, 501: 4, 251: 3, 492: 3, 227: 2, 390: 2, 415: 2, 425: 2, 89: 1, 121: 1, 194: 1, 221: 1, 223: 1, 276: 1, 293: 1, 451: 1, 458: 1, 577: 1, 635: 1}


In [47]:
unique_elements, counts = np.unique(closest_sv_index, return_counts=True)
df_counts = pd.DataFrame({'Index': unique_elements, 'Count': counts})
df_top5 = df_counts.sort_values(by='Count', ascending=False).head(5)
X_top5 = X_train.iloc[df_top5['Index'].values]

In [48]:
X_top5

,age,priors_count,days_b_screening_arrest,c_jail_time,juv_fel_count,juv_other_count,juv_misd_count,c_charge_degree:F,c_charge_degree:M,race:African-American,race:Asian,race:Caucasian,race:Hispanic,race:Native_American,race:Other,age_cat:25_-_45,age_cat:Greater_than_45,age_cat:Less_than_25,sex:Female,sex:Male
3520,31.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4257,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4106,39.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4390,37.0,2.0,-1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
875,28.0,3.0,0.0,-1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [49]:
test_df.iloc[explained_index:explained_index+1]

,age,priors_count,days_b_screening_arrest,c_jail_time,juv_fel_count,juv_other_count,juv_misd_count,c_charge_degree:F,c_charge_degree:M,race:African-American,race:Asian,race:Caucasian,race:Hispanic,race:Native_American,race:Other,age_cat:25_-_45,age_cat:Greater_than_45,age_cat:Less_than_25,sex:Female,sex:Male
4529,36.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [50]:
print(check_prediction_counterfactuals(est, query_instance, X_top5))
print(check_num_features_counterfactuals(query_instance, X_top5))
print(proximity_score(X_top5, query_instance, continuous_columns_idx))

1.0
3.2
(6.2, 1.2)


/accounts/projects/binyu/zhongyuan_liang/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/accounts/projects/binyu/zhongyuan_liang/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


### Match X & Groundtruth

In [51]:
test_row = test_df.iloc[explained_index:explained_index+1]
test_target = y_test[explained_index]
train_df_filtered = train_df[train_df['target'] != test_target].copy()
feature_columns = train_df.columns.drop('target')
test_values = test_row.iloc[0][feature_columns]
def count_feature_changes(row):
    return (row[feature_columns] != test_values).sum()
train_df_filtered['feature_changes'] = train_df_filtered.apply(count_feature_changes, axis=1)
closest_matches = train_df_filtered.nsmallest(5, 'feature_changes')

In [57]:
test_df.iloc[explained_index:explained_index+1]

,age,priors_count,days_b_screening_arrest,c_jail_time,juv_fel_count,juv_other_count,juv_misd_count,c_charge_degree:F,c_charge_degree:M,race:African-American,race:Asian,race:Caucasian,race:Hispanic,race:Native_American,race:Other,age_cat:25_-_45,age_cat:Greater_than_45,age_cat:Less_than_25,sex:Female,sex:Male
4529,36.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [52]:
closest_matches

,age,priors_count,days_b_screening_arrest,c_jail_time,juv_fel_count,juv_other_count,juv_misd_count,c_charge_degree:F,c_charge_degree:M,race:African-American,race:Asian,race:Caucasian,race:Hispanic,race:Native_American,race:Other,age_cat:25_-_45,age_cat:Greater_than_45,age_cat:Less_than_25,sex:Female,sex:Male,target,feature_changes
4257,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1,2
3324,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1,2
3520,31.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1,2
768,27.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1,2
143,25.0,1.0,-1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1,3


In [53]:
closest_matches_explanations = closest_matches.drop(columns=['target', 'feature_changes'])
print(check_prediction_counterfactuals(est, query_instance, closest_matches_explanations))
print(check_num_features_counterfactuals(query_instance, closest_matches_explanations))
print(proximity_score(closest_matches_explanations, query_instance, continuous_columns_idx))

1.0
2.2
(9.6, 0.6)


/accounts/projects/binyu/zhongyuan_liang/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/accounts/projects/binyu/zhongyuan_liang/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


### Match X & Prediciton

In [54]:
test_row = test_df.iloc[explained_index:explained_index+1]
train_predictions = est.predict(X_train)
test_target = y_test[explained_index]
train_df_filtered = train_df[train_predictions != test_target].copy()
feature_columns = train_df.columns.drop('target')
test_values = test_row.iloc[0][feature_columns]
train_df_filtered['feature_changes'] = train_df_filtered.apply(count_feature_changes, axis=1)
closest_matches = train_df_filtered.nsmallest(5, 'feature_changes')

In [58]:
test_df.iloc[explained_index:explained_index+1]

,age,priors_count,days_b_screening_arrest,c_jail_time,juv_fel_count,juv_other_count,juv_misd_count,c_charge_degree:F,c_charge_degree:M,race:African-American,race:Asian,race:Caucasian,race:Hispanic,race:Native_American,race:Other,age_cat:25_-_45,age_cat:Greater_than_45,age_cat:Less_than_25,sex:Female,sex:Male
4529,36.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [55]:
closest_matches

,age,priors_count,days_b_screening_arrest,c_jail_time,juv_fel_count,juv_other_count,juv_misd_count,c_charge_degree:F,c_charge_degree:M,race:African-American,race:Asian,race:Caucasian,race:Hispanic,race:Native_American,race:Other,age_cat:25_-_45,age_cat:Greater_than_45,age_cat:Less_than_25,sex:Female,sex:Male,target,feature_changes
4257,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1,2
3324,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1,2
3520,31.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1,2
768,27.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1,2
143,25.0,1.0,-1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1,3


In [56]:
closest_matches_explanations = closest_matches.drop(columns=['target', 'feature_changes'])
print(check_prediction_counterfactuals(est, query_instance, closest_matches_explanations))
print(check_num_features_counterfactuals(query_instance, closest_matches_explanations))
print(proximity_score(closest_matches_explanations, query_instance, continuous_columns_idx))

1.0
2.2
(9.6, 0.6)


/accounts/projects/binyu/zhongyuan_liang/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/accounts/projects/binyu/zhongyuan_liang/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
